In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from bleak import BleakScanner

print('Выберите тип датчика из списка')
detector_type = str(input('TD, TT, TL, DU, TH, TW, TP'))

async def run():
    devices = await BleakScanner.discover()  
    for device in devices:
        if devices is not None:
            # Ниже команда выбора датчика, туда можно вставить input
            filtered_devices = [(device.name, device.metadata, device.rssi) for device in devices if device.name is not None and 'TW' in device.name]
            for data in filtered_devices:
                print(data)
                pass
        
        print(device.name, device.metadata, device.rssi)
        #print(device.metadata)
        if device.name is not None:
            if device.name.startswith('TW'): 
                if 'manufacturer_data' in device.metadata:

                    data = device.metadata['manufacturer_data'][3862]

                    print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))

                    # Расшифровка уровня топлива
                    TW_oil_level_raw = int.from_bytes(data[1:3], byteorder='little', signed=False)
                    print()
                    print(f"Уровень топлива: {TW_oil_level_raw}")
                    
                    # 2 байт - не используемый байт, его не будем выводить
                    
                    # Расшифровка температуры
                    TW_temp_raw = int.from_bytes(data[4:5], byteorder='big', signed=True)
                    print(f"Температура: {TW_temp_raw} Градус Цельсия")

                    # Расшифровка версии прошивки
                    TW_version_raw = int.from_bytes(data[5:6], byteorder='big', signed=False)
                    print(f"Версия прошивки: {TW_version_raw}")
                    
                    # Расшифровка периода (непосредственно измеренный параметр)
                    TW_period_param = int.from_bytes(data[6:10], byteorder='little', signed=False)
                    print(f"Период: {TW_period_param} ")

                    # Расшифровка режима работы
                    TW_mode_raw = int.from_bytes(data[10:11], byteorder='big', signed=False)
                    TW_range = (TW_mode_raw & 0b11110000)
                    if TW_range & 0x80:
                        print("Диапазон датчика 4095")
                    if TW_range & 0b00000000:
                        print("Диапазон датчика 1024")

                    TW_flags = (TW_mode_raw & 0b00001111)
                    if TW_flags & 0b00000000:
                        print("Обычный алгоритм расчета")
                    if TW_flags & 0b00000001:
                        print("Алгоритм расчета с тарировочной таблицей")

                    # Расшифровка сглаживания и термокомпенсации
                    TW_smoth_and_copm_raw = int.from_bytes(data[11:12], byteorder='big', signed=False)
                    TW_therm_compensation = (TW_smoth_and_copm_raw & 0b11110000)
                    if TW_therm_compensation & 0x80:
                        print("Термокомпенсация выключена")
                    if TW_therm_compensation & 0b00000000:
                        print("Термокомпенсация включена")

                    TW_smoothing = TW_smoth_and_copm_raw & 0x0F
                    print(f'Сглаживание: {TW_smoothing}')
                    print('нужно проверить как работает расчет сглаживания, возможно я накосячил :)')
                    print()


loop = asyncio.get_event_loop()
loop.run_until_complete(run())